In [2]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Die Onkologie-Station

Auf einer Onkologie-Station liegen fünf Patienten in nebeneinander liegenden Zimmern.
Bis auf einen  der Patienten hat jeder genau eine Zigaretten-Marke geraucht.
Der Patient, der nicht Zigarette geraucht hat, hat Pfeife geraucht.
Jeder Patient fährt genau ein Auto und ist
an genau einer Krebs-Art erkrankt.  Zusätzlich haben Sie die folgenden Informationen:
<ol>
<li> Im Zimmer neben Michael wird Camel geraucht. </li>
<li> Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
      Zungen-Krebs Patienten. </li>
<li> Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. </li>
<li> Der West-Raucher liegt im ersten Zimmer. </li>
<li> Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. </li>
<li> Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. </li>
<li> Rudolf bettelt um Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. </li>
<li> Der Seat Fahrer hat morgen seinen letzten Geburtstag. </li>
<li> Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. </li>
<li> Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. </li>
<li> Der Nissan Fahrer liegt neben dem Mazda-Fahrer. </li>
<li> Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. </li>
<li> Jens liegt neben dem Luckies Raucher. </li>
<li> Der Hodenkrebs-Patient hat gestern seine Eier durchs Klo gespült. </li>
</ol>
Mit dieser Information können die beiden folgenden Fragen beantwortet werden.
<ol> 
<li> Was raucht der Darmkrebs-Patient? </li>
<li> Was fährt Kurt für ein Auto? </li>
</ol>

## Aufgabe

Formulieren Sie das oben beschrieben Rätsel als CSP und lösen Sie es mit Hilfe des Backtracking-Solvers, der in dem Notebook `Backtrack-Solver.ipynb` implementiert ist. 

In [3]:
def allDifferent(Variables: set[str]) -> set[str]:
    return { f'{x} != {y}' for x in Variables
                           for y in Variables 
                           if x < y 
           }

In [4]:
gCars = {'Trabant', 'Mazda', 'Nissan', 'Seat', 'Mercedes'}
gTabak = {'Pfeife', 'West', 'Camel', 'Luckies', 'Ernte23'}
gCancer = {'Zungen-Krebs', 'Lungen-Krebs', 'Hoden-Krebs', 'Kehlkopf-Krebs', 'Darm-Krebs'}
gNames = {'Kurt', 'Michael', 'Rudolf', 'Jens', 'Rolf'}
gRoomNumber = {'1', '2', '3', '4', '5'}

In [5]:
CSP = tuple[set[str] | list[str], set[int], set[str]]

In [6]:
def onkologie_csp() -> CSP: 
    Variables = gCars | gTabak | gCancer | gNames | gRoomNumber
    Values    = { 1, 2, 3, 4, 5 }
    Constraints = { 'abs(Camel - Michael) == 1',
                    'Trabant       == Ernte23',
                    'abs(Trabant - Zungen-Krebs) == 1',
                    'Rolf       == Kehlkopf-Krebs',
                    'Rolf == 5',
                    'West == 1',
                    'Mazda        == Zungen-Krebs', 
                    'abs(Mazda - Trabant) == 1',
                    'abs(Nissan - Zungen-Krebs) == 1',
                    'abs(Rudolf - Camel) == 1',
                    'abs(Rudolf - Trabant) == 1',
                    'abs(Luckies - Lungen-Krebs) == 1',
                    'abs(Camel - Darm-Krebs) == 1',
                    'abs(Nissan - Mazda) == 1',
                    'Mercedes == Pfeife',
                    'abs(Mercedes - Camel) == 1',
                    'abs(Jens - Luckies) == 1'
                  }
    Constraints |= allDifferent(gCars)
    Constraints |= allDifferent(gTabak)
    Constraints |= allDifferent(gCancer)
    Constraints |= allDifferent(gNames)
    Constraints |= allDifferent(gRoomNumber)
    return (Variables, Values, Constraints)

In [7]:
from typing import TypeVar

In [8]:
Value      = TypeVar('Value')
Element    = TypeVar('Element')
Variable   = str
Formula    = str
CSP        = tuple[set[Variable] | list[Variable], set[Value], set[Formula]]
ACSP       = tuple[set[Variable] | list[Variable], set[Value], list[tuple[Formula, set[Variable]]]]
Assignment = dict[Variable, Value]

In [9]:
def solve(P:            CSP, 
          debug:        bool=False, 
          animate:      bool=False, 
          problem_size: int|None=None) -> Assignment | None:
    Variables, Values, Constraints = P
    csp = (Variables, Values, [(f, collect_variables(f)) for f in Constraints])
    return backtrack_search({}, csp, debug, animate, problem_size)

In [10]:
def backtrack_search(Assgnmnt:     Assignment,
                     P:            ACSP, 
                     debug:        bool, 
                     animate:      bool, 
                     problem_size: int | None) -> Assignment | None:
    if debug and not animate:
        print(Assgnmnt)
    if animate:
        if problem_size == None:
            display(show_solution(Assgnmnt, width="50%")) # type: ignore
        else: 
            display(show_solution(Assgnmnt, problem_size, width="50%")) # type: ignore
    Variables, Values, Constraints = P
    if len(Assgnmnt) == len(Variables):
        return Assgnmnt
    if isinstance(Variables, set):
        var = arb(Variables - Assgnmnt.keys())
    else: # if Variables is a list we choose the first unassigned variable
        var = [x for x in Variables if x not in Assgnmnt][0]
    for value in Values:
        if is_consistent(var, value, Assgnmnt, Constraints):
            NewAss = Assgnmnt.copy()
            NewAss[var] = value
            Solution = backtrack_search(NewAss, P, debug, animate, problem_size)
            if Solution != None:
                return Solution
    return None

In [11]:
def is_consistent(var:         Variable, 
                  value:       Value, 
                  Assgnmnt:    Assignment, 
                  Constraints: list[tuple[Formula, set[Variable]]]) -> bool:
    NewAssign      = Assgnmnt.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if  var in Vs and Vs <= NewAssign.keys()
              )

In [12]:
import ast

In [13]:
def collect_variables(expression: Formula) -> set[Variable]: 
    tree = ast.parse(expression)
    return { node.id for node in ast.walk(tree) 
                     if  isinstance(node, ast.Name) 
                     if  node.id not in dir(__builtins__)
           }

In [14]:
def arb(S: set[Element]) -> Element:
    for x in S:
        return x
    return None # type: ignore

In [ ]:
onkologie = onkologie_csp()

In [ ]:
%%time
Solution = solve(onkologie)

In [ ]:
def show_solution(Solution: dict[str, set[int]]) -> None:
    result  = '<table style="border:2px solid blue">\n'
    result += '<tr>'
    for name in ['Name', 'Room',  'Krebs', 'Auto', 'Brand']:
        result += '<th style="color:gold; background-color:blue">' + name + '</th>'
    result += '</tr>\n'
    for chair in range(1, 5+1):
        result += '<tr><td style="border:1px solid green">' + str(chair) + '</td>'
        for Class in [gNames, gCancer, gTabak, gCars, gRoomNumber]:
            for x in Class:
                if Solution[x] == chair:
                    result += '<td  style="border:1px solid green">' + x + '</td>'
        result += '</tr>\n'
    result += '</table>'
    display(HTML(result)) # type: ignore

In [ ]:
show_solution(Solution)